# Baseline Algorithms for Forest Cover Type Predictions


Data source: https://www.kaggle.com/c/forest-cover-type-prediction

In [2]:
import numpy as np
import pandas as pd
import pickle

import seaborn as sns
import xgboost as xgb
import feature_eng_function as f_eng
import PCA

from datetime import datetime
from matplotlib import pyplot as plt
from PCA_function import pca_data100
from IPython.core.display import display, HTML
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV

%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ModuleNotFoundError: No module named 'PCA'

## Load Data and create base data set

In [2]:
forest = pd.read_csv("data/train.csv", index_col=0)
original_fields = forest.columns.tolist()
forest.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [3]:
data = forest.values
X_kaggle = data[:,0:(-1)]
y_train = data[:,-1]
X_kaggle.shape, y_train.shape

((15120, 54), (15120,))

## Feature Engineering

In [4]:
forest_eng = pd.read_csv("data/train_eng.csv", index_col=0)

Dropped the following columns: 

Wetmore
Pachic Argiborolis
Aquolis


In [5]:
forest_eng.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [6]:
data = forest_eng.values
X_base = data[:,0:(-1)]
y_train = data[:,-1]
X_base.shape, y_train.shape

((15120, 95), (15120,))

## Import the top 100 feature set data for testing

In [3]:
forest_100 = pd.read_csv("data/train_100.csv", index_col=0)
forest_100.head()

,Elevation,Horizontal_Distance_To_Hydrology_Hillshade_3pm,Horizontal_Distance_To_Roadways_Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Roadways_Aspect2,Hillshade_9am_Hillshade_Noon,Elevation_Elevation_3100_8000,Elevation_3100_8000_Elevation,Horizontal_Distance_To_Fire_Points_Horizontal_Distance_To_Roadways,Slope_Horizontal_Distance_To_Roadways,Slope_Horizontal_Distance_To_Fire_Points,...,Soil_Type39_Hillshade_9am,Wilderness_Area4_Elevation,Very Stony_Elevation,Wilderness_Area4_Hillshade_Noon,Outcrop_Horizontal_Distance_To_Roadways,Aspect2_Vanet,Rock_Elevation,Soil_Type32_Horizontal_Distance_To_Roadways,Horizontal_Distance_To_Roadways_Wilderness_Area4,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,38184,3202290,26010,51272,0,0,3202290,1530,18837,...,0,0,0.0,0,510.0,0.0,2596.0,0,0,5
2,2590,32012,2427750,21840,51700,0,0,2427750,780,12450,...,0,0,0.0,0,390.0,0.0,2590.0,0,0,5
3,2804,36180,19464780,442020,55692,0,0,19464780,28620,55089,...,0,0,0.0,0,0.0,0.0,0.0,0,0,2
4,2785,29524,19191990,478950,56644,0,0,19191990,55620,111798,...,0,0,0.0,0,0.0,0.0,0.0,0,0,2
5,2595,22950,2413252,17595,51480,0,0,2413252,782,12344,...,0,0,0.0,0,391.0,0.0,2595.0,0,0,5


In [9]:
data = forest_100.values
X_100 = data[:,0:(-1)]
y_train = data[:,-1]
X_100.shape, y_train.shape

((15120, 100), (15120,))

## Run XGBoost in Grid Search across a broad range of parameters for each data set

In [184]:
models = []

In [92]:
X_kaggle_train, X_kaggle_test, y_kaggle_train, y_kaggle_test = train_test_split(X_kaggle, y_train)
optimized_kaggle_default_GBM = xgb.XGBClassifier(seed=0,objective='multi:softmax')
optimized_kaggle_default_GBM.fit(X_kaggle_train, y_kaggle_train, eval_metric="merror",
    eval_set=[(X_kaggle_test, y_kaggle_test)])

[0]	validation_0-merror:0.395767
[1]	validation_0-merror:0.380423
[2]	validation_0-merror:0.33545
[3]	validation_0-merror:0.337037
[4]	validation_0-merror:0.34127
[5]	validation_0-merror:0.330688
[6]	validation_0-merror:0.330159
[7]	validation_0-merror:0.325397
[8]	validation_0-merror:0.326455
[9]	validation_0-merror:0.324868
[10]	validation_0-merror:0.319577
[11]	validation_0-merror:0.319048
[12]	validation_0-merror:0.313228
[13]	validation_0-merror:0.31455
[14]	validation_0-merror:0.314815
[15]	validation_0-merror:0.31455
[16]	validation_0-merror:0.313228
[17]	validation_0-merror:0.310582
[18]	validation_0-merror:0.30873
[19]	validation_0-merror:0.303968
[20]	validation_0-merror:0.301587
[21]	validation_0-merror:0.30582
[22]	validation_0-merror:0.300529
[23]	validation_0-merror:0.297884
[24]	validation_0-merror:0.298413
[25]	validation_0-merror:0.297354
[26]	validation_0-merror:0.296032
[27]	validation_0-merror:0.294709
[28]	validation_0-merror:0.294974
[29]	validation_0-merror:0.293

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [185]:
models.append(optimized_kaggle_default_GBM)

In [10]:
t1 = datetime.now()
cv_params = {'max_depth': [1,3,5], 'min_child_weight': [1,3,5], 'learning_rate': [0.15, 0.1, 0.075, 0.05]}
ind_params = {'n_estimators': 300, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'multi:softmax'}
optimized_kaggle_cv5_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, scoring = 'accuracy', cv = 5)#, n_jobs = -1)
optimized_kaggle_cv5_GBM.fit(X_kaggle, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 5360 sec


In [186]:
models.append(optimized_kaggle_cv5_GBM)

NameError: name 'optimized_kaggle_cv5_GBM' is not defined

In [11]:
optimized_kaggle_cv5_GBM.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.69061, std: 0.03408, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 1},
 mean: 0.69061, std: 0.03355, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 3},
 mean: 0.68909, std: 0.03482, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 5},
 mean: 0.75099, std: 0.03659, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 1},
 mean: 0.75218, std: 0.03708, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 3},
 mean: 0.75073, std: 0.03496, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 5},
 mean: 0.77864, std: 0.03479, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1},
 mean: 0.77447, std: 0.03964, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 3},
 mean: 0.77282, std: 0.03644, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 5},
 mean: 0.67870, std: 0.03251, params: {'learning_rate': 0.1, 'max_depth': 1, 'min_

In [64]:
t1 = datetime.now()
cv_params = {'max_depth': [1,3,5], 'min_child_weight': [1,3,5], 'learning_rate': [0.15, 0.1, 0.075, 0.05]}
ind_params = {'n_estimators': 300, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'multi:softmax'}
optimized_kaggle_cv10_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, scoring = 'accuracy', cv = 10, n_jobs = -1)
optimized_kaggle_cv10_GBM.fit(X_kaggle, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 8247 sec


In [187]:
models.append(optimized_kaggle_cv10_GBM)

In [65]:
optimized_kaggle_cv10_GBM.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.69319, std: 0.03904, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 1},
 mean: 0.69272, std: 0.03909, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 3},
 mean: 0.69153, std: 0.04162, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 5},
 mean: 0.75198, std: 0.04160, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 1},
 mean: 0.75040, std: 0.04303, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 3},
 mean: 0.74570, std: 0.04296, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 5},
 mean: 0.77778, std: 0.03824, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1},
 mean: 0.77374, std: 0.04127, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 3},
 mean: 0.76971, std: 0.04470, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 5},
 mean: 0.68624, std: 0.03856, params: {'learning_rate': 0.1, 'max_depth': 1, 'min_

In [91]:
X_base_train, X_base_test, y_base_train, y_base_test = train_test_split(X_base, y_train)
optimized_base_default_GBM = xgb.XGBClassifier(seed=0,objective='multi:softmax')
optimized_base_default_GBM.fit(X_base_train, y_base_train, eval_metric="merror",
    eval_set=[(X_base_test, y_base_test)])

[0]	validation_0-merror:0.349735
[1]	validation_0-merror:0.340212
[2]	validation_0-merror:0.346561
[3]	validation_0-merror:0.33836
[4]	validation_0-merror:0.334127
[5]	validation_0-merror:0.321693
[6]	validation_0-merror:0.321429
[7]	validation_0-merror:0.318783
[8]	validation_0-merror:0.31746
[9]	validation_0-merror:0.316402
[10]	validation_0-merror:0.312963
[11]	validation_0-merror:0.308466
[12]	validation_0-merror:0.308995
[13]	validation_0-merror:0.30873
[14]	validation_0-merror:0.306878
[15]	validation_0-merror:0.305556
[16]	validation_0-merror:0.303175
[17]	validation_0-merror:0.300529
[18]	validation_0-merror:0.302646
[19]	validation_0-merror:0.299735
[20]	validation_0-merror:0.298413
[21]	validation_0-merror:0.297884
[22]	validation_0-merror:0.298148
[23]	validation_0-merror:0.298942
[24]	validation_0-merror:0.296032
[25]	validation_0-merror:0.29709
[26]	validation_0-merror:0.296032
[27]	validation_0-merror:0.294974
[28]	validation_0-merror:0.293651
[29]	validation_0-merror:0.2

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [188]:
models.append(optimized_base_default_GBM)

In [12]:
t1 = datetime.now()
cv_params = {'max_depth': [1,3,5], 'min_child_weight': [1,3,5], 'learning_rate': [0.15, 0.1, 0.075, 0.05]}
ind_params = {'n_estimators': 300, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'multi:softmax'}
optimized_base_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, scoring = 'accuracy', cv = 5, n_jobs = -1)
optimized_base_GBM.fit(X_base, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 3754 sec


In [13]:
optimized_base_GBM.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.69034, std: 0.03449, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 1},
 mean: 0.69180, std: 0.03370, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 3},
 mean: 0.69220, std: 0.03422, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 5},
 mean: 0.75516, std: 0.03795, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 1},
 mean: 0.75344, std: 0.03834, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 3},
 mean: 0.74934, std: 0.03857, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 5},
 mean: 0.78095, std: 0.03316, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1},
 mean: 0.77573, std: 0.03491, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 3},
 mean: 0.77698, std: 0.03563, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 5},
 mean: 0.68439, std: 0.03199, params: {'learning_rate': 0.1, 'max_depth': 1, 'min_

In [ ]:
t1 = datetime.now()
cv_params = {'max_depth': [1,3,5], 'min_child_weight': [1,3,5], 'learning_rate': [0.15, 0.1, 0.075, 0.05]}
ind_params = {'n_estimators': 300, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'multi:softmax'}
optimized_base_cv10_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, scoring = 'accuracy', cv = 10, n_jobs = -1)
optimized_base_cv10_GBM.fit(X_base, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

In [ ]:
models.append(optimized_base_cv10_GBM)

In [ ]:
optimized_base_cv10_GBM.grid_scores_

In [190]:
X_100_train, X_100_test, y_100_train, y_100_test = train_test_split(X_100, y_train)
optimized_100_default_GBM = xgb.XGBClassifier(seed=0,objective='multi:softmax')
optimized_100_default_GBM.fit(X_100_train, y_100_train, early_stopping_rounds=50, eval_metric="merror",
    eval_set=[(X_100_test, y_100_test)])

optimized_100_default_GBM_pred = optimized_100_default_GBM.predict(X_100_test)
optimized_100_default_GBM_accuracy = accuracy_score(y_100_test, optimized_100_default_GBM_pred);
print ('accuracy:%0.2f%%'%(optimized_100_default_GBM_accuracy*100))

[0]	validation_0-merror:0.317725
Will train until validation_0-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.317196
[2]	validation_0-merror:0.306878
[3]	validation_0-merror:0.31455
[4]	validation_0-merror:0.304233
[5]	validation_0-merror:0.309524
[6]	validation_0-merror:0.30291
[7]	validation_0-merror:0.30291
[8]	validation_0-merror:0.30582
[9]	validation_0-merror:0.30582
[10]	validation_0-merror:0.301058
[11]	validation_0-merror:0.297354
[12]	validation_0-merror:0.293122
[13]	validation_0-merror:0.291799
[14]	validation_0-merror:0.287302
[15]	validation_0-merror:0.288889
[16]	validation_0-merror:0.287037
[17]	validation_0-merror:0.283333
[18]	validation_0-merror:0.28254
[19]	validation_0-merror:0.281481
[20]	validation_0-merror:0.281746
[21]	validation_0-merror:0.280159
[22]	validation_0-merror:0.279101
[23]	validation_0-merror:0.27672
[24]	validation_0-merror:0.276984
[25]	validation_0-merror:0.276984
[26]	validation_0-merror:0.27619
[27]	validation_0-merror:0.275661

In [191]:
models.append(optimized_100_default_GBM)

In [14]:
t1 = datetime.now()
cv_params = {'max_depth': [1,3,5], 'min_child_weight': [1,3,5], 'learning_rate': [0.15, 0.1, 0.075, 0.05]}
ind_params = {'n_estimators': 300, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'multi:softmax'}
optimized_100_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, scoring = 'accuracy', cv = 5, n_jobs = -1)
optimized_100_GBM.fit(X_100, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 4903 sec


In [192]:
models.append(optimized_100_GBM)

In [94]:
optimized_100_GBM.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.70060, std: 0.03620, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 1},
 mean: 0.70132, std: 0.03721, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 3},
 mean: 0.70033, std: 0.03655, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 5},
 mean: 0.77500, std: 0.03620, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 1},
 mean: 0.77295, std: 0.03565, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 3},
 mean: 0.76951, std: 0.03670, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 5},
 mean: 0.79788, std: 0.03663, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1},
 mean: 0.79239, std: 0.03677, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 3},
 mean: 0.79187, std: 0.03674, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 5},
 mean: 0.69008, std: 0.03703, params: {'learning_rate': 0.1, 'max_depth': 1, 'min_

In [ ]:
t1 = datetime.now()
cv_params = {'max_depth': [1,3,5], 'min_child_weight': [1,3,5], 'learning_rate': [0.15, 0.1, 0.075, 0.05]}
ind_params = {'n_estimators': 300, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'multi:softmax'}
optimized_100_cv10_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, scoring = 'accuracy', cv = 10, n_jobs = -1)
optimized_100_cv10_GBM.fit(X_100, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

In [ ]:
models.append(optimized_100_cv10_GBM)

In [ ]:
optimized_100_cv10_GBM.grid_scores_

## Fine Tune the best hyperparameters for the best model in the broad search

In [16]:
############ NEEDS TO BE MODIFIED ONCE WE FIGURE OUT THE BEST ONE ####################
t1 = datetime.now()
cv_params = {'max_depth': [1,3,5], 'min_child_weight': [1,3,5], 'learning_rate': [0.15, 0.1, 0.075, 0.05]}
ind_params = {'n_estimators': 300, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'multi:softmax'}
optimized_kaggle_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), cv_params, scoring = 'accuracy', cv = 5)#, n_jobs = -1)
optimized_kaggle_GBM.fit(X_kaggle, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.70060, std: 0.03620, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 1},
 mean: 0.70132, std: 0.03721, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 3},
 mean: 0.70033, std: 0.03655, params: {'learning_rate': 0.15, 'max_depth': 1, 'min_child_weight': 5},
 mean: 0.77500, std: 0.03620, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 1},
 mean: 0.77295, std: 0.03565, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 3},
 mean: 0.76951, std: 0.03670, params: {'learning_rate': 0.15, 'max_depth': 3, 'min_child_weight': 5},
 mean: 0.79788, std: 0.03663, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1},
 mean: 0.79239, std: 0.03677, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 3},
 mean: 0.79187, std: 0.03674, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 5},
 mean: 0.69008, std: 0.03703, params: {'learning_rate': 0.1, 'max_depth': 1, 'min_

## Random Forest Ensembles in Grid Search

In [40]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_kaggle_cv3_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=3, n_jobs=-1)
optimized_kaggle_cv3_RF.fit(X_kaggle, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 383 sec


In [193]:
models.append(optimized_kaggle_cv3_RF)

In [41]:
optimized_kaggle_cv3_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.40602, std: 0.02613, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.42553, std: 0.02691, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.43909, std: 0.04771, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.44134, std: 0.06432, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.45317, std: 0.04335, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.45384, std: 0.04278, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.45754, std: 0.04227, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.46138, std: 0.03837, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.44180, std: 0.0

In [42]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_kaggle_cv5_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=5, n_jobs=-1)
optimized_kaggle_cv5_RF.fit(X_kaggle, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 707 sec


In [194]:
models.append(optimized_kaggle_cv5_RF)

In [43]:
optimized_kaggle_cv5_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.45231, std: 0.01502, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.45549, std: 0.01143, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.47910, std: 0.01698, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.46852, std: 0.03014, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.46786, std: 0.03025, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.48776, std: 0.02424, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.48942, std: 0.02423, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.48823, std: 0.02366, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.47454, std: 0.0

In [44]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_kaggle_cv10_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=10, n_jobs=-1)
optimized_kaggle_cv10_RF.fit(X_kaggle, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 1496 sec


In [195]:
models.append(optimized_kaggle_cv10_RF)

In [45]:
optimized_kaggle_cv10_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.44974, std: 0.04753, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.46561, std: 0.05157, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.48426, std: 0.04744, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.48228, std: 0.05189, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.48307, std: 0.05616, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.48406, std: 0.05590, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.48697, std: 0.05471, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.48724, std: 0.05301, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.47930, std: 0.0

In [52]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_base_cv3_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=3, n_jobs=-1)
optimized_base_cv3_RF.fit(X_base, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 409 sec


In [196]:
models.append(optimized_base_cv3_RF)

In [53]:
optimized_base_cv3_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.48294, std: 0.01013, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.48340, std: 0.03176, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.45721, std: 0.01603, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.48373, std: 0.01714, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.49352, std: 0.01741, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.49517, std: 0.01982, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.49464, std: 0.02074, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.49848, std: 0.01776, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.53075, std: 0.0

In [54]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_base_cv5_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=5, n_jobs=-1)
optimized_base_cv5_RF.fit(X_base, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 753 sec


In [197]:
models.append(optimized_base_cv5_RF)

In [55]:
optimized_base_cv3_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.48294, std: 0.01013, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.48340, std: 0.03176, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.45721, std: 0.01603, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.48373, std: 0.01714, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.49352, std: 0.01741, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.49517, std: 0.01982, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.49464, std: 0.02074, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.49848, std: 0.01776, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.53075, std: 0.0

In [56]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_base_cv10_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=10, n_jobs=-1)
optimized_base_cv10_RF.fit(X_base, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 1634 sec


In [198]:
models.append(optimized_base_cv10_RF)

In [57]:
optimized_base_cv10_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.47903, std: 0.04816, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.45615, std: 0.03975, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.45417, std: 0.04226, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.45622, std: 0.04208, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.46541, std: 0.04609, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.46329, std: 0.04603, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.47784, std: 0.05587, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.47745, std: 0.04982, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.53188, std: 0.0

In [58]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_100_cv3_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=3, n_jobs=-1)
optimized_100_cv3_RF.fit(X_100, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 786 sec


In [199]:
models.append(optimized_100_cv3_RF)

In [59]:
optimized_100_cv3_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.43909, std: 0.01309, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.43929, std: 0.01372, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.43876, std: 0.01318, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.44015, std: 0.01408, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.44484, std: 0.01858, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.44497, std: 0.01861, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.44398, std: 0.01764, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.44405, std: 0.01761, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.47269, std: 0.0

In [60]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_100_cv5_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=5, n_jobs=-1)
optimized_100_cv5_RF.fit(X_100, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 1559 sec


In [200]:
models.append(optimized_100_cv5_RF)

In [61]:
optimized_100_cv5_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.44028, std: 0.01134, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.43836, std: 0.01526, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.44306, std: 0.01158, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.44292, std: 0.01184, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.44597, std: 0.01461, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.44980, std: 0.01564, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.44332, std: 0.01112, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.44306, std: 0.01108, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.48505, std: 0.0

In [62]:
t1 = datetime.now()
hyperparameters = {'max_depth': [1,3,5],
    'n_estimators': [50, 100, 200, 300, 500, 700, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}
optimized_100_cv10_RF = GridSearchCV(RandomForestClassifier(random_state=0), hyperparameters, cv=10, n_jobs=-1)
optimized_100_cv10_RF.fit(X_100, y_train)
t2 = datetime.now()
print ("Total time: %i sec" %(t2-t1).seconds)

Total time: 3335 sec


In [201]:
models.append(optimized_100_cv10_RF)

In [63]:
optimized_100_cv10_RF.grid_scores_

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.42778, std: 0.01130, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 50},
 mean: 0.42976, std: 0.01405, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 100},
 mean: 0.43419, std: 0.01498, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 200},
 mean: 0.43439, std: 0.01484, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 300},
 mean: 0.43611, std: 0.01576, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 500},
 mean: 0.43591, std: 0.01511, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 700},
 mean: 0.43552, std: 0.01488, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 800},
 mean: 0.43565, std: 0.01494, params: {'criterion': 'gini', 'max_depth': 1, 'max_features': 'sqrt', 'n_estimators': 1000},
 mean: 0.46356, std: 0.0

## Save model fits to pickle so I can stop using all of my computer's memory

In [204]:
import pickle

In [207]:
models[0]

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [209]:
for i in range(len(models)):
    name = str('pickles/model'+str(i+1))
    with open(name, 'wb') as f:
        pickle.dump(models[i], f)
    f.close()
    

In [210]:
with open('pickles/model1', 'rb') as f:
    m1 = pickle.load(f)

In [211]:
m1.predict(test_X)

array([ 2.,  2.,  1., ...,  3.,  3.,  3.])

## Try base classifiers for PCA

In [109]:
from PCA_function import pca_data100
forest_pca = forest_eng
cover_type = forest_pca['Cover_Type']
forest_pca = forest_pca[[f for f in forest_pca.columns.tolist() if f != 'Cover_Type']]
forest_pca = pca_data100(forest_pca)
forest_pca.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,4.385772e+06,-1.480827e+07,-7.125796e+06,-5.801071e+06,281747.013864,1.251481e+06,689101.982964,144790.705239,-293407.928603,-16671.201528,...,98.607957,-1772.132576,-1137.189310,-129.921202,1481.605603,-1112.677633,-128.922789,381.872618,1263.076570,1141.376467
1,3.215416e+06,-1.471615e+07,-7.171142e+06,-6.371079e+06,214636.464538,1.290512e+06,268567.823683,53440.577448,-310100.308796,-26774.762836,...,222.637067,-1851.963792,-1227.263724,37.497726,1544.950396,-1212.174072,-88.975996,319.605714,1298.807692,1041.685859
2,2.699070e+07,-1.272132e+07,1.967986e+06,1.338300e+06,-2025.581086,1.165906e+03,74474.050902,669382.150682,326874.213567,-92597.666495,...,-353.726531,560.859349,1873.402241,911.800567,-109.259301,-675.948023,-73.726350,-677.370050,1108.011486,542.922156
3,2.654173e+07,-1.309087e+07,1.853466e+06,1.110555e+06,-132664.666300,7.340862e+03,-53554.941589,684225.733947,872612.789452,-84904.028765,...,412.939213,-656.489382,-1768.906761,-55.109699,70.532150,836.551667,244.722875,-409.746252,315.886056,1010.340382
4,3.066968e+06,-1.458155e+07,-6.709171e+06,-6.686220e+06,277160.748671,1.273784e+06,-249333.807449,-196178.564477,-288300.991274,-59794.386702,...,470.733306,-1831.865001,-985.513302,-206.217047,1632.210306,-1220.630371,120.039911,300.698859,1435.812193,985.793766


In [110]:
X_pca = forest_pca.values
y_pca = cover_type.values

In [112]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y_pca)
optimized_pca_default_GBM = xgb.XGBClassifier(seed=0,objective='multi:softmax')
optimized_pca_default_GBM.fit(X_pca_train, y_pca_train, early_stopping_rounds=10, eval_metric="merror",
    eval_set=[(X_pca_test, y_pca_test)])

[0]	validation_0-merror:0.493915
Will train until validation_0-merror hasn't improved in 10 rounds.
[1]	validation_0-merror:0.459524
[2]	validation_0-merror:0.439418
[3]	validation_0-merror:0.433862
[4]	validation_0-merror:0.421164
[5]	validation_0-merror:0.407672
[6]	validation_0-merror:0.400794
[7]	validation_0-merror:0.406878
[8]	validation_0-merror:0.391799
[9]	validation_0-merror:0.384656
[10]	validation_0-merror:0.384921
[11]	validation_0-merror:0.379365
[12]	validation_0-merror:0.376984
[13]	validation_0-merror:0.370106
[14]	validation_0-merror:0.362434
[15]	validation_0-merror:0.365344
[16]	validation_0-merror:0.357407
[17]	validation_0-merror:0.35582
[18]	validation_0-merror:0.351323
[19]	validation_0-merror:0.349735
[20]	validation_0-merror:0.349471
[21]	validation_0-merror:0.344709
[22]	validation_0-merror:0.342593
[23]	validation_0-merror:0.341005
[24]	validation_0-merror:0.335714
[25]	validation_0-merror:0.331746
[26]	validation_0-merror:0.329365
[27]	validation_0-merror:0

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

## Create predictions with the best models
#### Using:
   - optimized_100_GBM: mean: 0.79788, std: 0.03663, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1}
   - optimized_kaggle_GBM: mean: 0.79788, std: 0.03663, params: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1}

In [157]:
test = pd.read_csv('data/test.csv', header=0)
test.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,15121,2680,354,14,0,0,2684,196,214,156,...,0,0,0,0,0,0,0,0,0,0
1,15122,2683,0,13,0,0,2654,201,216,152,...,0,0,0,0,0,0,0,0,0,0
2,15123,2713,16,15,0,0,2980,206,208,137,...,0,0,0,0,0,0,0,0,0,0
3,15124,2709,24,17,0,0,2950,208,201,125,...,0,0,0,0,0,0,0,0,0,0
4,15125,2706,29,19,0,0,2920,210,195,115,...,0,0,0,0,0,0,0,0,0,0


In [159]:
test_X = test[[t for t in test.columns.tolist() if t != 'Id']].values
test_X.shape

(565892, 54)

In [160]:
y_test_kaggle_GBM = optimized_kaggle_GBM.predict(test_X)

In [161]:
y_test_kaggle_GBM[:5]

array([ 2.,  1.,  1.,  1.,  1.])

In [162]:
y_test_kaggle_GBM.shape, test.Id.shape

((565892,), (565892,))

In [168]:
results_kaggle_GBM = pd.DataFrame(test.Id)
results_kaggle_GBM['Cover_Type'] = y_test_kaggle_GBM
results_kaggle_GBM['Cover_Type'] = results_kaggle_GBM['Cover_Type'].astype(int)
results_kaggle_GBM.to_csv('submissions/results_kaggle_GBM.csv', index=False)
#make sure you manually delete the last line
# SCORE = 0.73838

In [167]:
results_kaggle_GBM.head()

,Id,Cover_Type
0,15121,2
1,15122,1
2,15123,1
3,15124,1
4,15125,1


In [179]:
def d_feature_eng_forest(data_file_path, soil_file_path, test=False):
	try:
		df = pd.read_csv(data_file_path, sep=',', header=0, index_col='Id')
		soil_types = pd.read_csv(soil_file_path).set_index('Soil Type')
	except:
		df = pd.read_csv(data_file_path, sep=',', header=0)
		soil_types = pd.read_csv(soil_file_path).set_index('Soil Type')
	def labelSoilType(row):
		for i in range(len(row)):
			if row[i] == 1:
				return 'Soil_Type'+str(i)
	df['Direct_Distance_To_Hydrology']=np.sqrt((df.Vertical_Distance_To_Hydrology**2) + (df.Horizontal_Distance_To_Hydrology**2)).astype(float).round(2)
	def azimuth_to_abs(x):
		if x>180:
			return 360-x
		else:
			return x

    # Create Soil Type Buckets
	soil_types = pd.read_csv('soil_types.csv').set_index('Soil Type')
	df['Soil Type'] = df[['Soil_Type1', 'Soil_Type2', 'Soil_Type3',
           'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7',
           'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
           'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15',
           'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19',
           'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23',
           'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27',
           'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31',
           'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
           'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
           'Soil_Type40']].apply(lambda row: labelSoilType(row), axis=1)
	df = pd.merge(df, soil_types, how='left', left_on='Soil Type', right_index=True)
	del df['Soil Type'] # Delete string column

	# Create feature to that transforms azimuth to its absolute value
	df['Aspect2'] = df.Aspect.map(azimuth_to_abs)
	df['Aspect2'].astype(int)

	# Create feature that determines if the patch is above sea level
	df['Above_Sealevel'] = (df.Vertical_Distance_To_Hydrology>0).astype(int)

	# Bin the Elevation Feature: check the feature exploration notebook for motivation
	bins = [0, 2600, 3100, 8000]
	group_names = [1, 2, 3]
	df['Elevation_Bucket'] = pd.cut(df['Elevation'], bins, labels=group_names)
	df['Elevation_0_2600'] = np.where(df['Elevation_Bucket']== 1, 1, 0)
	df['Elevation_2600_3100'] = np.where(df['Elevation_Bucket']== 2, 1, 0)
	df['Elevation_3100_8000'] = np.where(df['Elevation_Bucket']== 3, 1, 0)
	df['Elevation_0_2600'].astype(int)
	df['Elevation_2600_3100'].astype(int)
	df['Elevation_3100_8000'].astype(int)
	del df['Elevation_Bucket']

	# Create a feature for no hillshade at 3pm
	df['3PM_0_Hillshade'] = (df.Hillshade_3pm == 0).astype(int)

	#Direct distance to hydrology
	df['Direct_Distance_To_Hydrology'] = np.sqrt((df.Vertical_Distance_To_Hydrology**2) + \
        (df.Horizontal_Distance_To_Hydrology**2)).astype(float).round(2)


	soil_types= ['Soil_Type1', 'Soil_Type2', 'Soil_Type3',
           'Soil_Type4', 'Soil_Type5', 'Soil_Type6',
           'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
           'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
           'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19',
           'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23',
           'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27',
           'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31',
           'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35',
           'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
           'Soil_Type40', 'Cover_Type']

	column_list = df.columns.tolist()
	column_list = [c for c in column_list if c[:9] != 'Soil_Type']
	column_list.insert(10, 'Direct_Distance_To_Hydrology')
	column_list.insert(11, 'Elevation_0_2600')
	column_list.insert(12, 'Elevation_2600_3100')
	column_list.insert(13, 'Elevation_3100_8000')
	column_list.insert(14, 'Aspect2')
	column_list.insert(15, 'Above_Sealevel')
	column_list.insert(16, '3PM_0_Hillshade')
	column_list.extend(soil_types)
	columns = []
	for col in column_list:
		if col not in columns:
			if col != 'Cover_Type':
				columns.append(col)
	if test:
		pass
	else:
		columns.append('Cover_Type')

	df = df[columns]
	df.fillna(0,inplace=True) # Replace nans with 0 for our soil type bins

	if not test:    
		to_remove = [] # features to drop
		for c in df.columns.tolist():
			if df[c].std() == 0:
				to_remove.append(c)
		df = df.drop(to_remove, 1)
		print("Dropped the following columns: \n")
		for r in to_remove:
			print (r)
        
	return df

def d_forest_interactions(df, test=False): 
	if test:
		for i in range(df.shape[1]):
			for j in range(54):
				if i != j:
					df[df.columns.tolist()[i]+"_"+df.columns.tolist()[j]] = df[df.columns.tolist()[i]]*df[df.columns.tolist()[j]]
	else:
		for i in range(df.shape[1]-1):
			for j in range(54):
				if i != j:
					df[df.columns.tolist()[i]+"_"+df.columns.tolist()[j]] = df[df.columns.tolist()[i]]*df[df.columns.tolist()[j]]

	if not test:    
		to_remove = [] # features to drop
		for c in df.columns.tolist():
			if df[c].std() == 0:
				to_remove.append(c)
		df = df.drop(to_remove, 1)
		print("Dropped the following columns: \n")
		for r in to_remove:
			print (r)
            
	return df


In [183]:
#import feature_eng_function as f_eng
eng = f_eng.feature_eng_forest('data/train.csv', 'soil_types.csv')
test_eng_X = test_X
test_eng_X = d_feature_eng_forest('data/test.csv','soil_types.csv', test=True)
test_eng_X = test_eng_X[[f for f in eng.columns.tolist() if f != 'Cover_Type']]
test_100_X = d_forest_interactions(test_eng_X, test=True)
test_100_X = test_100_X[top_100_features]

Dropped the following columns: 

Wetmore
Pachic Argiborolis
Aquolis
Dropped the following columns: 



KeyboardInterrupt: 

In [173]:
import feature_eng_function as f_eng
test_eng_X = test_X
test_eng_X = f_eng.feature_eng_forest('data/test.csv','soil_types.csv', test=True)
test_100_X = f.eng.forest_interactions(test_eng_X)
test_100_X = test_100_X[top_100_features]

TypeError: feature_eng_forest() got an unexpected keyword argument 'test'

In [ ]:
y_test_100_GBM = optimized_100_GBM.predict(test_100_X)
results_100_GBM = pd.DataFrame(test.Id)
results_100_GBM['Cover_Type'] = y_test_100_GBM
results_100_GBM['Cover_Type'] = results_kaggle_GBM['Cover_Type'].astype(int)
results_100_GBM.to_csv('submissions/results_100_GBM.csv', index=False)
#make sure you manually delete the last line
# SCORE = 

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points',
       ...
       'Soil_Type40_Limber', 'Soil_Type40_Troutville', 'Soil_Type40_Bullwark',
       'Soil_Type40_Catamount', 'Soil_Type40_Legault.1', 'Soil_Type40_Land',
       'Soil_Type40_Cryaquolis', 'Soil_Type40_Cryoborolis',
       'Soil_Type40_Gateview', 'Soil_Type40_Rogert'],
      dtype='object', length=5172)